In [46]:

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Masking
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import torch


In [109]:
df=pd.read_csv('./data/data_3.csv')

In [124]:
def mape(A,F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
   
    if maskf_sub[i]!=0:
      sum+=abs(A[i] - F[i]) / abs(A[i])
      length+=1
  if length>0:
    return 100/length*sum
  
  return 0
def smape(A, F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
    if maskf_sub[i]!=0:
      sum+=2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))
      length+=1
  if length>0:

    return 100/length * sum
  return 0


def get_mask(data):
    """
    data should in the form of pd.df
    gen a tenor with 0 and 1 to represent missing data
    """
    mask = ~data.isna().values
 
    mask_tensor = torch.tensor(mask, dtype=torch.float32)
    
    mask_tensor= mask_tensor.transpose(0,1)
    return mask_tensor


def normalize(data):
    first_list=[]
    for i in data.columns:
       
        first=data[i].iloc[0]
        p=1
        
        while np.isnan(first):
           first=data[i].iloc[p]

           p+=1
        
        data[i]=(data[i])/first
        first_list.append(first)   
    return data,first_list

In [ ]:

def lstm_run(df,name,ep,j):
    """df is the data in pd.DataFrame
    name is the list of columns name to be predicted
    ep is the epoch to train
    j is the prediction length"""    
    
    X = df[name][:-1]
    Y = df[name][1:]
  

    X_norm,X_div =normalize(X) 
    Y_norm,Y_div =normalize(Y)
    Y_test_mask=get_mask(Y_norm)[:,-j:]
    X_norm=X_norm.fillna(0)
    Y_norm=Y_norm.fillna(0)
    X_norm=X_norm.values
    Y_norm=Y_norm.values

    X_train, Y_train = X_norm[:-j], Y_norm[:-j]
    X_test, Y_test = X_norm[-j:], Y_norm[-j:]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(Masking(mask_value=0,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2]))  
    model.compile(loss='mae', optimizer='adam')
    model.fit(X_train, Y_train, epochs=ep, batch_size=32, validation_data=(X_test, Y_test), verbose=2, shuffle=False)

    test_predict = model.predict(X_test)
    smpl=[]
    mpl=[]
    pred=[]
    test=[]
    Y_test=Y_test[-j:].transpose(1,0)
    test_predict=test_predict[-j:].transpose(1,0)


    for i in range(len(df)):
        mp=mape(Y_test[i]*Y_div[i],test_predict[i]*Y_div[i],Y_test_mask[i])
        smp=smape(Y_test[i]*Y_div[i],test_predict[i]*Y_div[i],Y_test_mask[i])
        pred.append(test_predict[i]*Y_div[i])
        test.append(Y_test[i]*Y_div[i])
        smpl.append(smp)
        mpl.append(mp)
    return pred,test,mpl,smpl

In [ ]:
ep=20
j=-7
name=df.columns[1:]
pred,test,mpl,smpl=lstm_run(df,name,ep,j)


In [ ]:
result= pd.DataFrame({'lstm_mape':mpl,'lstm_smape':smpl})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':name,'day1/7_pred':pred[0], 'day1/7_actu':test[0], 'day2/7_pred':pred[1], 'day2/7_actu':test[1],
                       'day3/7_pred':pred[2], 'day3/7_actu':test[2], 'day4/7_pred':pred[3], 'day4/7_actu':test[3], 
                       'day5/7_pred':pred[4], 'day5/7_actu':test[4], 'day6/7_pred':pred[5], 'day6/7_actu':test[5],
                       'day7/7_pred':pred[6], 'day7/7_actu':test[6]
                       })
res2.to_csv('D:/timeseries/result/temp.csv')